# Profiling

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lukeconibear/swd6_hpp/blob/main/docs/01_profiling.ipynb)

In [ ]:
# if you're using colab, then install the required modules
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    %pip install line_profiler snakeviz pyinstrument eliot eliot-tree

[Profiling](https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html) analyses your code in terms of speed or memory. 

This helps identify where the bottlenecks are and how much potential there is for improvement.

_Side note_: [IPython magic commands](https://jakevdp.github.io/PythonDataScienceHandbook/01.03-magic-commands.html) are useful, commonly used commands in Jupyter Notebooks.  

- They have one `%` at the start for use on a single line.
- Or two `%%` at the start for use over a whole cell.

For example:

## [`timeit`](https://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)

A IPython magic command that measures the time execution of an expression. 

It runs a few times, depending on how intensive the expression is, and returns the best result with the uncertainty.

It is useful for benchmarking a small code snippet.

In [1]:
%timeit range(100)

63.8 ns ± 0.283 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [2]:
%%timeit
for x in range(100):
    pass

720 ns ± 5.57 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


Another option is:

## [`line_profiler`](https://github.com/pyutils/line_profiler)

A module that measures the time spent in each line of a function.

In [3]:
%load_ext line_profiler

In [6]:
import time

def relax():
    pass

def bottleneck():
    time.sleep(0.001)

def some_function():
    nums = [num for num in range(1_000)]
    bigger_nums = [num ** 2 for num in nums]
    bottleneck()
    bigger_nums.extend(bigger_nums)
    relax()

- The `Timer unit` is the conversion to seconds.
- The table shows the following columns for each line of the function (each row):
    - `Line #`: The line number.
    - `Hits`: The number of times that line was called.
    - `Time`: The total time spent calling that line (in the `Timer units`).
    - `Per Hit`: The time spend for one call of that line.
    - `% Time`: The percentage of total time spent on that line (this is the often the most useful column).
    - `Line Contents`: The source code for that line.

In [7]:
%lprun -f some_function some_function()

Timer unit: 1e-06 s

Total time: 0.001427 s
File: /tmp/ipykernel_1040/572293778.py
Function: some_function at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def some_function():
    10         1         74.0     74.0      5.2      nums = [num for num in range(1_000)]
    11         1        248.0    248.0     17.4      bigger_nums = [num ** 2 for num in nums]
    12         1       1101.0   1101.0     77.2      bottleneck()
    13         1          3.0      3.0      0.2      bigger_nums.extend(bigger_nums)
    14         1          1.0      1.0      0.1      relax()

We can see that most of the time was in the `bottleneck()` call, highlighting the potential here for possible improvements.

## [`SnakeViz`](https://jiffyclub.github.io/snakeviz/)

`SnakeViz` visualises the output from `cProfile` in nice interactive plots.

`cProfile` is part of the Python standard library.

It's a deterministic profiler of wallclock time (giving the same output for the same input).

However, the output has lots of tabular information that isn't ideal. Every function call gets measured in seconds (each row).

For example:

In [ ]:
import cProfile

In [ ]:
cProfile.run('some_function()')

Visualising this output in `SnakeViz` is nicer.

You can switch between the two styles using the "Style" dropdown.

### Style: Icicle

- Functions are represented by rectangles.
- The root function is the _top-most rectangle_, with functions it calls below it, then the functions those call below them, and so on.
- The amount of time spent inside a function is represented by the width of the rectangle.
    - A big rectangle that stretches across most of the visualization represents a function that is taking up most of the time of its calling function, while a small rectangle represents a function that is using hardly any time at all.

### Style: Sunburst

- Functions are represented by arcs.
- The root function is the _centre circle_, with functions it calls around, then the functions those functions call, and so on. 
- The amount of time spent inside a function is represented by how far around the circle the arc goes.
    - An big arc that wraps most of the way around the circle represents a function that is taking up most of the time of its calling function, while a small arc represents a function that is using hardly any time at all.

In [ ]:
%load_ext snakeviz

In [ ]:
%%snakeviz
some_function()

Note, `SnakeViz` doesn't currently work with Colab ([issue](https://github.com/jiffyclub/snakeviz/issues/164)).

## [`pyinstrument`](https://pyinstrument.readthedocs.io/en/latest/)

A statistical profiler of wall-clock time (recording the call stack every 1ms), lowering the overhead compared to tracing profilers (like `cProfile`).

It hides library frames and fast functions, so you can focus on the slow parts of your code.

The output shows a tree of actions of *how* the function executes, using a traffic light colour legend (i.e., red = slow).

In [ ]:
%load_ext pyinstrument

In [ ]:
%%pyinstrument
some_function()

## [`Eliot`](https://eliot.readthedocs.io/en/stable/index.html)

Logs the performance information. 

It tracks each function called, its arguments, the return values, and any rasied exceptions.

Hence, it can be helpful for profiling in real use cases.

Note, that this logging has a higher overhead than `cProfile`.

It works by using decorators, which are a nice way for functions to return other functions.

For example, here is one that logs calls to a function:

In [ ]:
def log_calls(function):
    
    def wrapper():
        name = function.__name__
        
        print(f'Before {name} was called ...')
        function()
        print(f'After {name} was called ...')
        
    return wrapper

Calling this returns a function:

In [ ]:
log_calls(some_function)

So, to execute it we add `()`:

In [ ]:
log_calls(some_function)()

The `@wrapper` syntax is a nice way to return this same result, where the outer function is placed above the inner function:

In [ ]:
@log_calls
def some_function_with_logging():
    some_function()

In [ ]:
some_function_with_logging()

Okay, so now we can use the `@eliot.log_call` decorator to profile a function:

In [ ]:
import eliot

In [ ]:
def bottleneck():
    time.sleep(0.5)

@eliot.log_call
def multiply(x, y):
    bottleneck()
    return x * y

@eliot.log_call
def add(x, y):
    return x + y

# tell eliot where to log the output messages
eliot.to_file(open('profiling_some_function.log', 'a'))
result = multiply(add(3, 4), add(5, 6))

In [ ]:
!eliot-tree profiling_some_function.log

[Aside: Visualising how functions work in Python](https://pythontutor.com/composingprograms.html#code=def%20multiply%28x,%20y%29%3A%0A%20%20%20%20return%20x%20*%20y%0A%0Adef%20add%28x,%20y%29%3A%0A%20%20%20%20return%20x%20%2B%20y%0A%0Amultiply%28add%283,%204%29,%20add%285,%206%29%29&cumulative=true&curInstr=0&mode=display&origin=composingprograms.js&py=3&rawInputLstJSON=%5B%5D)

## Exercise

...

## Further information

### Other options

- [`VizTracer`](https://github.com/gaogaotiantian/viztracer)
  - A low-overhead logging/debugging/profiling tool that can trace and visualize your python code execution.
- [Using `eliot` with Dask](https://eliot.readthedocs.io/en/stable/scientific-computing.html)
- [Profile parallel code with Dask](https://docs.dask.org/en/latest/diagnostics-local.html#example)  

#### Memory profiling

- [`memory_profiler`](https://github.com/pythonprofilers/memory_profiler)
  - Measures the memory used by a function, at its peak and the overall increment.
  
### Resources

- [Beyond `cProfile`: Choosing the right tool for performance optimization](https://pythonspeed.com/articles/beyond-cprofile/)